# * ETL-Actual MKS : Broadband
    (P, Z, G, H, HH, CCAA) level

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re
import FN_Actual_MKS_Broadband as fn

config = configparser.ConfigParser()
config.read('../../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
next_week_dt = curr_dt + dt.timedelta(days=7)
str_curr_dt = curr_dt.strftime('%Y%m%d')
str_next_week_dt = next_week_dt.strftime('%Y%m%d')
yyyymm_curr = curr_dt.strftime('%Y%m')
yyyymm_next_week = next_week_dt.strftime('%Y%m')

### Latest Source Summary

In [31]:
# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()


try:
    # Get : Actual Summary from "FCT_BB_SHAREV4_ISP_SUBS"
    src_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(COALESCE(TOL,0) + COALESCE(BBB,0) + COALESCE(AIS,0) + COALESCE(NT,0)) TOTAL
            , SUM(TOL) TOL
            , SUM(BBB) "3BB"
            , SUM(AIS) AIS
            , SUM(NT) NT
        FROM CORPNSBOX.FCT_BB_SHAREV4_ISP_SUBS A
        WHERE TM_KEY_MTH >= 202401
        AND VER = 'FINAL'
        AND AREA_TYPE = 'CCAA'
        AND EXISTS (SELECT 1 FROM CDSAPPO.DIM_MOOC_AREA O
                    WHERE O.TEAM_CODE <> 'ไม่ระบุ' AND O.REMARK <> 'Dummy'
                    AND SUBSTR(O.CCAATT,1,4) = A.AREA_CD)
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = src_cur.fetchall()
    print(f'\nCurrent Source Summary...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    tmp_src_df = chk_src_df.copy()
    mod_col_list = tmp_src_df.iloc[:, 1:7].columns.tolist()
    for col in mod_col_list:
        tmp_src_df[col] = tmp_src_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{tmp_src_df}')

    # Get : MAX(TM_KEY_MTH)
    v_mth_end_src = chk_src_df['TM_KEY_MTH'].max().astype(float)
    print(f'\nv_mth_end_src: {v_mth_end_src}')
    # print(f'\nv_mth_end_src: {v_mth_end_src[0]}')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Current Source Summary...

    TM_KEY_MTH       TOTAL        TOL        3BB        AIS         NT
0       202401   9,610,606  3,062,599  2,172,204  2,575,793  1,800,010
1       202402   9,637,448  3,066,654  2,146,854  2,623,938  1,800,002
2       202403   9,667,104  3,073,225  2,119,793  2,674,081  1,800,005
3       202404   9,662,022  3,073,401  2,105,276  2,683,359  1,799,986
4       202405   9,698,572  3,082,684  2,135,762  2,680,120  1,800,006
5       202406   9,763,878  3,089,418  2,186,591  2,687,880  1,799,989
6       202407   9,815,121  3,099,649  2,210,652  2,704,836  1,799,984
7       202408   9,825,568  3,103,283  2,197,805  2,724,485  1,799,995
8       202409   9,855,554  3,111,079  2,200,915  2,743,557  1,800,003
9       202410   9,846,986  3,119,072  2,185,528  2,742,384  1,800,002
10      202411   9,890,763  3,124,727  2,230,269  2,735,754  1,800,013
11      202412   9,916,000  3,132,060  2,242,448  2,741,487  1,800,005
12      202501   9,975,280  3,141,442  2,013,932 

### Latest Fact Summary

In [32]:
# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
tgt_cur = tgt_conn.cursor()


try:
    # Get : Actual Summary from "FCT_BROADBAND_MKS"
    tgt_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(CASE WHEN METRIC_CD IN ('VIN00081', 'VIN00082', 'VIN00083', 'VIN00084') THEN SUBS_VALUE END) TOTAL
            , SUM(CASE WHEN METRIC_CD = 'VIN00081' THEN SUBS_VALUE END) TOL
            , SUM(CASE WHEN METRIC_CD = 'VIN00082' THEN SUBS_VALUE END) "3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00083' THEN SUBS_VALUE END) AIS
            , SUM(CASE WHEN METRIC_CD = 'VIN00084' THEN SUBS_VALUE END) NT
            --, MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.FCT_BROADBAND_MKS 
        WHERE TM_KEY_MTH >= 202401 --AND TM_KEY_MTH <= 202404
        AND TM_KEY_DAY LIKE '%01'
        AND AREA_TYPE = 'P'
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = tgt_cur.fetchall()
    print(f'\nCurrent Fact Summary...')
    chk_tgt_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])

    # Display
    tmp_tgt_df = chk_tgt_df.copy()
    mod_col_list = tmp_tgt_df.iloc[:, 1:6].columns.tolist()
    for col in mod_col_list:
        tmp_tgt_df[col] = tmp_tgt_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{tmp_tgt_df}')

    # Get : MAX(TM_KEY_MTH)
    v_mth_end_fct = chk_tgt_df['TM_KEY_MTH'].max().astype(float)
    print(f'\nv_mth_end_fct: {v_mth_end_fct}')

    # tgt_cur.execute("SELECT MAX(TM_KEY_MTH) FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK")
    # max_mth_fct = tgt_cur.fetchone()
    # print(f'\nv_max_mth_fct: {max_mth_fct[0]}')
    
    tgt_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()


Current Fact Summary...

    TM_KEY_MTH       TOTAL        TOL        3BB        AIS         NT
0       202401   9,610,606  3,062,599  2,172,204  2,575,793  1,800,010
1       202402   9,637,448  3,066,654  2,146,854  2,623,938  1,800,002
2       202403   9,667,104  3,073,225  2,119,793  2,674,081  1,800,005
3       202404   9,662,022  3,073,401  2,105,276  2,683,359  1,799,986
4       202405   9,698,572  3,082,684  2,135,762  2,680,120  1,800,006
5       202406   9,763,878  3,089,418  2,186,591  2,687,880  1,799,989
6       202407   9,815,121  3,099,649  2,210,652  2,704,836  1,799,984
7       202408   9,825,568  3,103,283  2,197,805  2,724,485  1,799,995
8       202409   9,855,554  3,111,079  2,200,915  2,743,557  1,800,003
9       202410   9,846,986  3,119,072  2,185,528  2,742,384  1,800,002
10      202411   9,890,763  3,124,727  2,230,269  2,735,754  1,800,013
11      202412   9,916,000  3,132,060  2,242,448  2,741,487  1,800,005
12      202501   9,975,280  3,141,442  2,013,932  3

### Check Diff Summary

In [31]:
# ''' Temp '''

# chk_diff_df = chk_src_df.iloc[-1].compare(chk_tgt_df.iloc[-1])
# chk_diff_df['diff'] = chk_diff_df['self'] - chk_diff_df['other']
# mod_col_list = chk_diff_df.columns.tolist()
# for col in mod_col_list:
#     chk_diff_df[col] = chk_diff_df[col].apply(lambda x: format(x, ',.0f'))

# chk_diff_df

In [33]:
''' Compare Latest Actual '''

latest_src_actual = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==v_mth_end_src]
# latest_src_actual
latest_fct_actual = chk_tgt_df.loc[chk_tgt_df['TM_KEY_MTH']==v_mth_end_src]
# latest_fct_actual

chk_diff_latest_actual = latest_src_actual.iloc[-1].compare(latest_fct_actual.iloc[-1])
chk_diff_latest_actual['diff'] = chk_diff_latest_actual['self'] - chk_diff_latest_actual['other']
mod_col_list = chk_diff_latest_actual.columns.tolist()
for col in mod_col_list:
    chk_diff_latest_actual[col] = chk_diff_latest_actual[col].apply(lambda x: format(x, ',.0f'))

chk_diff_latest_actual

,self,other,diff


### Input Parameter

In [35]:
''' Manual Config '''
# v_update_flag = 'N'
v_target_schema = 'AUTOKPI'
v_target_table = 'FCT_BROADBAND_MKS'
v_sql_update_fact = 'Update-FCT_BROADBAND_MKS.sql'
v_sql_mockup_fact = 'Mock-FCT_BROADBAND_MKS.sql'
v_sql_initial_fact = 'Initial-FCT_BROADBAND_MKS.sql'

''' Auto Config '''
v_update_flag = 'Y' if chk_diff_latest_actual.size > 0 else 'N'
# v_max_mth_fct = max_mth_fct[0]
v_mth_end_fct_year = pd.to_datetime(v_mth_end_fct, format='%Y%m').year
v_mth_end_fct_month = pd.to_datetime(v_mth_end_fct, format='%Y%m').month
v_prev_mth_fct = v_mth_end_fct-1 if v_mth_end_fct_month != 1 else float(str(v_mth_end_fct_year-1)+str(12))

''' Show Auto Config '''
print(f'\nyyyymm_next_week: {yyyymm_next_week}')
print(f'\nv_update_flag: {v_update_flag}')
print(f'v_mth_end_src: {v_mth_end_src}')
print(f'v_mth_end_fct: {v_mth_end_fct}')
# print(f'v_max_mth_fct: {v_max_mth_fct}')
print(f'v_prev_mth_fct: {v_prev_mth_fct}')


yyyymm_next_week: 202505

v_update_flag: N
v_mth_end_src: 202503.0
v_mth_end_fct: 202504.0
v_prev_mth_fct: 202503.0


## ETL Process...

### DB source to DB Target
    Delete -> Insert

    Source : CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT
             CDSAPPO.DIM_MOOC_AREA
             CDSAPPO.DIM_TIME
    
    Target : AUTOKPI.FCT_BROADBAND_MKS

#### Incremental

In [36]:
''' Auto Process '''

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')

# Process flow
if v_update_flag == 'Y':
    if yyyymm_next_week > str(v_mth_end_fct):
        print(f'\n*** Source update & mockup to next month Fact ***')
        fn.src_update_to_fact(v_mth_end_src, v_target_schema, v_target_table, v_sql_update_fact)
    else:
        print(f'\n*** Source update current month Fact ***')
        fn.src_update_to_fact(v_prev_mth_fct, v_target_schema, v_target_table, v_sql_update_fact)

elif v_update_flag == 'N':
    if yyyymm_next_week > str(v_mth_end_fct):
        print(f'\n*** Last Fact mockup to next month ***')
        fn.mockup_to_fact(v_mth_end_fct, v_mth_end_src, v_target_schema, v_target_table, v_sql_mockup_fact)
    else:
        print(f'\n*** Not update ***')
        
print(f'\nJob Done !!!')


Job Start... 2025-04-28, 15:12:15

*** Last Fact mockup to next month ***

Param input...

   -> mth_end_fct: 202504.0
   -> mth_end_src: 202503.0
   -> target_schema: AUTOKPI
   -> target_table: FCT_BROADBAND_MKS
   -> sql_mockup_fact: Mock-FCT_BROADBAND_MKS.sql
   -> v_query_param: {'mth_end_fct': 202504.0, 'mth_end_src': 202503.0}

AKPIPRD : Connected

Processing...

   -> DELETE : "FCT_BROADBAND_MKS" : Done !

   -> INSERT : "FCT_BROADBAND_MKS" : Done !

AKPIPRD : Disconnected

Job Done !!!


#### Initial

In [11]:
''' Initial Process '''

# v_initial_mth_start = 202401
# v_initial_mth_end = 202412

# job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# print(f'\nJob Start... {job_start_datetime}')

# print(f'\n*** Re-run initial data to Fact ***')
# fn.src_initial_to_fact(v_initial_mth_start, v_initial_mth_end, v_target_schema, v_target_table, v_sql_initial_fact)

# print(f'\nJob Done !!!')


Job Start... 2025-02-11, 12:33:09

*** Re-run initial data to Fact ***

Param input...

   -> initial_mth_start: 202401
   -> initial_mth_end: 202412
   -> target_schema: AUTOKPI
   -> target_table: FCT_BROADBAND_MKS
   -> sql_initial_fact: Initial-FCT_BROADBAND_MKS.sql
   -> v_query_param: {'initial_mth_start': 202401, 'initial_mth_end': 202412}

TDMDBPR : Connected

AKPIPRD : Connected

Processing...

Create Dataframe...

   -> src_df : 4306230 rows, 17 columns

   -> DELETE : "FCT_BROADBAND_MKS" : Done !

   -> INSERT : "FCT_BROADBAND_MKS" : Done !

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


## Check Result : "FCT_BROADBAND_MKS"

In [ ]:
''' Version 2024 : Create Result DataFrame '''

# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
tgt_cur = tgt_conn.cursor()
# print(f'\n{AKPIPRD_db} : Connected')


try:
    # Get : Result Data Summary
    tgt_cur.execute("""
        SELECT TM_KEY_MTH, AREA_NO, AREA_TYPE, COALESCE(REMARK, 'Actual') REMARK
            -->> Subs
            , SUM(CASE WHEN METRIC_CD IN ('VIN00025', 'VIN00026', 'VIN00027', 'VIN00028') THEN METRIC_VALUE END) TOTAL
            , SUM(CASE WHEN METRIC_CD = 'VIN00024' THEN METRIC_VALUE END) "AIS & 3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00025' THEN METRIC_VALUE END) TOL
            , SUM(CASE WHEN METRIC_CD = 'VIN00026' THEN METRIC_VALUE END) "3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00027' THEN METRIC_VALUE END) AIS
            , SUM(CASE WHEN METRIC_CD = 'VIN00028' THEN METRIC_VALUE END) NT
            -->> % MKS
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD IN ('VIN00020', 'VIN00021', 'VIN00022', 'VIN00023') THEN METRIC_VALUE END) "% TOTAL"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00019' THEN METRIC_VALUE END) "% AIS & 3BB"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00020' THEN METRIC_VALUE END) "% TOL"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00021' THEN METRIC_VALUE END) "% 3BB"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00022' THEN METRIC_VALUE END) "% AIS"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00023' THEN METRIC_VALUE END) "% NT"
            -->> Count
            , COUNT(DISTINCT METRIC_CD) CNT_METRIC, COUNT(1) ROW_CNT, MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK
        WHERE TM_KEY_DAY LIKE '%01'
        GROUP BY TM_KEY_MTH, AREA_NO, AREA_TYPE, REMARK
        ORDER BY 1,2
    """)
    rows = tgt_cur.fetchall()
    print(f'\nGet : Fact Summary...')
    chk_result_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])
    print(f'\n   -> chk_result_df : {chk_result_df.shape[0]} rows, {chk_result_df.shape[1]} columns') 
    
    # Display
    tmp_result_df = chk_result_df.copy()
    # tmp_result_df = tmp_result_df.replace(np.nan, None)
    # tmp_result_df.iloc[:, 4:18] = tmp_result_df.iloc[:, 4:18].fillna(0)
    mod_col_list = tmp_result_df.iloc[:, 4:18].columns.tolist()
    for col in mod_col_list:
        tmp_result_df[col] = tmp_result_df[col].apply(lambda x: format(x, ',.2f') if re.search('%', col) else format(x, ',.0f'))
        
    tgt_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()

In [37]:
''' Create Result DataFrame '''

# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
tgt_cur = tgt_conn.cursor()


try:
    # Get : Result Data Summary
    tgt_cur.execute("""
        SELECT TM_KEY_MTH, AREA_NO, AREA_TYPE, COALESCE(REMARK, 'Actual') REMARK
            -->> Subs
            , SUM(CASE WHEN METRIC_CD IN ('VIN00086', 'VIN00087', 'VIN00088', 'VIN00089') THEN METRIC_VALUE END) TOTAL
            , SUM(CASE WHEN METRIC_CD = 'VIN00085' THEN METRIC_VALUE END) "AIS & 3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00086' THEN METRIC_VALUE END) TOL
            , SUM(CASE WHEN METRIC_CD = 'VIN00087' THEN METRIC_VALUE END) "3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00088' THEN METRIC_VALUE END) AIS
            , SUM(CASE WHEN METRIC_CD = 'VIN00089' THEN METRIC_VALUE END) NT
            -->> % MKS
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD IN ('VIN00081', 'VIN00082', 'VIN00083', 'VIN00084') THEN METRIC_VALUE END) "% TOTAL"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00080' THEN METRIC_VALUE END) "% AIS & 3BB"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00081' THEN METRIC_VALUE END) "% TOL"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00082' THEN METRIC_VALUE END) "% 3BB"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00083' THEN METRIC_VALUE END) "% AIS"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00084' THEN METRIC_VALUE END) "% NT"
            -->> Count
            , COUNT(DISTINCT METRIC_CD) CNT_METRIC, COUNT(1) ROW_CNT, MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK
        WHERE TM_KEY_DAY LIKE '%01'
        GROUP BY TM_KEY_MTH, AREA_NO, AREA_TYPE, REMARK
        --ORDER BY 1,2
    """)
    rows = tgt_cur.fetchall()
    print(f'\nGet : Fact Summary...')
    chk_result_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])
    print(f'\n   -> chk_result_df : {chk_result_df.shape[0]} rows, {chk_result_df.shape[1]} columns') 
    
    # Display
    tmp_result_df = chk_result_df.copy()
    # tmp_result_df = tmp_result_df.replace(np.nan, None)
    # tmp_result_df.iloc[:, 4:18] = tmp_result_df.iloc[:, 4:18].fillna(0)
    mod_col_list = tmp_result_df.iloc[:, 4:18].columns.tolist()
    for col in mod_col_list:
        tmp_result_df[col] = tmp_result_df[col].apply(lambda x: format(x, ',.2f') if re.search('%', col) else format(x, ',.0f'))
        
    tgt_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()


Get : Fact Summary...

   -> chk_result_df : 170 rows, 19 columns


In [38]:
''' Group by '''

agg_df = chk_result_df.groupby(['REMARK', 'TM_KEY_MTH', 'CNT_METRIC']).agg({'AREA_TYPE': 'count', 'ROW_CNT': 'sum', 'LOAD_DATE': 'max'}).sort_values(by=['TM_KEY_MTH']).reset_index()
agg_df

,REMARK,TM_KEY_MTH,CNT_METRIC,AREA_TYPE,ROW_CNT,LOAD_DATE
0,Actual,202401,10,10,11800,2025-02-11 12:39:05.666791
1,Actual,202402,10,10,11800,2025-02-11 12:39:05.666791
2,Actual,202403,10,10,11800,2025-02-11 12:39:05.666791
3,Actual,202404,10,10,11800,2025-02-11 12:39:05.666791
4,Actual,202405,10,10,11800,2025-02-11 12:39:05.666791
5,Actual,202406,10,10,11800,2025-02-11 12:39:05.666791
6,Actual,202407,10,10,11740,2025-02-11 12:39:05.666791
7,Actual,202408,10,10,11730,2025-02-11 12:39:05.666791
8,Actual,202409,10,10,11730,2025-02-11 12:39:05.666791
9,Actual,202410,10,10,11730,2025-02-11 12:39:05.666791


In [39]:
''' Reconcile '''

# Filter
max_mth = chk_result_df['TM_KEY_MTH'].max()
# rec_df = tmp_result_df.loc[tmp_result_df['TM_KEY_MTH']==max_mth]
rec_df = tmp_result_df.loc[tmp_result_df['AREA_TYPE']=='P']
rec_df = rec_df.loc[rec_df['TM_KEY_MTH']>=202410]
# rec_df = rec_df.loc[rec_df['TM_KEY_MTH']<=202501]
rec_df = rec_df.sort_values(by=['TM_KEY_MTH']).reset_index(drop=True)

rec_df#.tail(3)

,TM_KEY_MTH,AREA_NO,AREA_TYPE,REMARK,TOTAL,AIS & 3BB,TOL,3BB,AIS,NT,% TOTAL,% AIS & 3BB,% TOL,% 3BB,% AIS,% NT,CNT_METRIC,ROW_CNT,LOAD_DATE
0,202410,1,P,Actual,"9,846,986","4,927,888","3,119,072","2,185,528","2,742,384","1,800,002",100.00,50.04,31.68,22.19,27.85,18.28,10,10,2025-02-11 12:39:05.666791
1,202411,1,P,Actual,"9,890,763","4,966,042","3,124,727","2,230,269","2,735,754","1,800,013",100.00,50.21,31.59,22.55,27.66,18.20,10,10,2025-02-11 12:39:05.666791
2,202412,1,P,Actual,"9,916,000","4,983,931","3,132,060","2,242,448","2,741,487","1,800,005",100.00,50.26,31.59,22.61,27.65,18.15,10,10,2025-02-11 12:39:05.666791
3,202501,1,P,Actual,"9,975,280","5,033,861","3,141,442","2,013,932","3,019,915","1,799,991",100.00,50.46,31.49,20.19,30.27,18.04,10,10,2025-02-11 12:42:04.146650
4,202502,1,P,Actual,"10,036,258","5,086,763","3,149,505","1,936,550","3,150,202","1,800,001",100.00,50.68,31.38,19.30,31.39,17.93,10,10,2025-03-15 16:26:31.000000
5,202503,1,P,Actual,"10,014,376","5,051,692","3,162,673","1,911,413","3,140,289","1,800,001",100.00,50.44,31.58,19.09,31.36,17.97,10,10,2025-04-08 10:51:38.668977
6,202504,1,P,Data as of : 202503,"10,014,376","5,051,692","3,162,673","1,911,413","3,140,289","1,800,001",100.00,50.44,31.58,19.09,31.36,17.97,10,10,2025-04-08 10:51:38.668977
7,202505,1,P,Data as of : 202503,"10,014,376","5,051,692","3,162,673","1,911,413","3,140,289","1,800,001",100.00,50.44,31.58,19.09,31.36,17.97,10,10,2025-04-28 15:13:00.488631
